In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
# neural network(using keras)
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.backend import expand_dims

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
def build_model_single_layer(layer, model, dilation_rate=None):
    os.chdir(r'/home/tdeng/PBMC/tree-cnn/train/layer'+str(layer))
    print(os.listdir())
    
    train_features = np.load('features.npy')
    train_features = expand_dims(train_features, axis=-1).numpy()
    
    train_labels = np.load('labels.npy')
    print((train_labels == 0).sum()/train_labels.shape[0])
    print((train_labels == 1).sum()/train_labels.shape[0])
    print((train_labels == 2).sum()/train_labels.shape[0])
    cell_type_num = np.unique(train_labels).shape[0]
    train_labels = to_categorical(train_labels)

#     X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.99,
#                                                         random_state=2020, shuffle=True)
#     X_train = expand_dims(X_train, axis=-1).numpy()
#     X_test = expand_dims(X_test, axis=-1).numpy()
#     y_train = to_categorical(y_train)
#     y_test = to_categorical(y_test)
    
    
    
    if model == 'cnn' and dilation_rate == None:
        print("please specify the parameter 'dilation_rate'.")
        return None
    elif model == 'cnn' and dilation_rate != None:
        #开始构建模型
        model = Sequential()
        model.add(Conv1D(32, 7,dilation_rate=dilation_rate, activation='relu', input_shape=(32738,1)))
        model.add(MaxPooling1D(5))
        model.add(Conv1D(64, 5, activation='relu'))
        model.add(Flatten())
        model.add(Dense(cell_type_num, activation='softmax'))
        #输出摘要
        model.summary()
        #编译模型
        model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        if layer != 4:
            model.fit(train_features, train_labels, epochs=1, batch_size=128)
        else:
            model.fit(train_features, train_labels, epochs=3, batch_size=128)
        model.save('normal_cnn.h5')
        


In [3]:
def build_model_all_layers(model, dilation_rate=None):
    for i in range(1, 5):
        build_model_single_layer(i, model, dilation_rate)

build_model_all_layers('cnn', dilation_rate=5)

['.ipynb_checkpoints', 'features.npy', 'labels.npy']
0.9881163202535185
0.0118836797464815
0.0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 32708, 32)         256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6541, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 6537, 64)          10304     
_________________________________________________________________
flatten_1 (Flatten)          (None, 418368)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 836738    
Total params: 847,298
Trainable params: 847,298
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
42916/429